In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B-Instruct", torch_dtype=torch.float32, device_map="auto")
pipe(messages)

شغال شات كتب ع المكعالج 1

In [2]:
# ipython-input-25-bccdb090aded

#!pip install pymupdf sentence-transformers transformers

import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
# Instead of AutoModelForSeq2SeqLM, import AutoModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np

# Load the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the Llama model and tokenizer
llama_model_name = "meta-llama/Llama-3.1-8B-Instruct"
# Use AutoModelForCausalLM to load the Llama model
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_name, torch_dtype=torch.float32, device_map="auto")
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_name)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Function to generate embeddings
def generate_embeddings(text):
    sentences = text.split('. ')
    embeddings = embedding_model.encode(sentences)
    return sentences, embeddings

# Path to your PDF file
pdf_path = '/content/Xenobi-Amilen.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
sentences, embeddings = generate_embeddings(pdf_text)

# Save the embeddings and sentences for later use
np.save('embeddings.npy', embeddings)
with open('sentences.txt', 'w') as f:
    for sentence in sentences:
        f.write("%s\n" % sentence)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

شغال شات كتب ع المكعالج 2

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Load embeddings and sentences
embeddings = np.load('embeddings.npy')
with open('sentences.txt', 'r') as f:
    sentences = f.readlines()

# Function to find the most similar sentence
def find_similar_sentence(query, embeddings):
    query_embedding = embedding_model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    most_similar_idx = np.argmax(similarities)
    return sentences[most_similar_idx]

# Function to generate response using Llama model
def generate_response(query):
    similar_sentence = find_similar_sentence(query, embeddings)
    input_text = f"Q: {query}\nA: {similar_sentence}"
    inputs = llama_tokenizer(input_text, return_tensors="pt")
    outputs = llama_model.generate(**inputs)
    response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Simple chat interface
while True:
    user_query = input("You: ")
    if user_query.lower() in ["exit", "quit"]:
        break
    response = generate_response(user_query)
    print(f"Llama: {response}")